In [17]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.metrics import confusion_matrix

# Baseline model
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')
maxLen = len(max(X_train, key=len).split())#because we have to embed 0 for other short sentences

Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)


word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

def sentence_to_avg(sentence, word_to_vec_map):
	words = (sentence.lower()).split()
	avg = np.zeros([50,])#because word_to_vec_map is 50-dimensional
	for w in words:
		avg += word_to_vec_map[w]
		avg = avg/len(words)

	return avg

def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
	np.random.seed(1)
	m = Y.shape[0]
	n_y = 5
	n_h = 50

	W = np.random.randn(n_y, n_h) / np.sqrt(n_h) # initialization
	b = np.zeros((n_y,))

	#convert Y to Y_onehot
	Y_oh = convert_to_one_hot(Y, C=n_y)

	for t in range(num_iterations):
		for i in range(m):
			avg = sentence_to_avg(X[i], word_to_vec_map)

			z = np.dot(W, avg) + b
			a = softmax(z) # same shape with Y_oh

			cost = -np.sum(Y_oh[i]*np.log10(a))

			dz = a - Y_oh[i]
			dW = np.dot(dz.reshape(n_y, 1), avg.reshape(1, n_h))
			db = dz

			W = W - learning_rate * dW
			b = b - learning_rate * db


		if t%100 == 0:
			print("Epoch: " + str(t) + " --- cost = " + str(cost))
			pred = predict(X, Y, W, b, word_to_vec_map)
	return pred, W, b

ped, W, b = model(X_train, Y_train, word_to_vec_map)
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

In [18]:
#Test
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.8333333333333334

i adore you ❤️
i love you ❤️
funny lol 😄
lets play with a ball ⚾
food is ready 🍴
not feeling happy ❤️


In [32]:
# USIng LSTM in keras
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

def sentences_to_indices(X, word_to_index, max_len):
	m = X.shape[0]
	X_indices = np.zeros([m, max_len])
	for i in range(m):
		sentence_words = (X[i].lower()).split()
		j = 0
		for w in sentence_words:
			X_indices[i, j] = word_to_index[w]
			j = j + 1

	return X_indices
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
	vocab_len = len(word_to_index) + 1
	emb_dim = word_to_vec_map["cucumber"].shape[0]
	emb_matrix = np.zeros([vocab_len, emb_dim])
	for word, index in word_to_index.items():
		emb_matrix[index, :] = word_to_vec_map[word]
	#define keras embedding layer
	embedding_layer = Embedding(vocab_len, emb_dim, 
		embeddings_initializer='uniform', trainable = False)
	# has to use this build before set_weights
	embedding_layer.build((None,))
	# kind of initialization
	embedding_layer.set_weights([emb_matrix])

	return embedding_layer

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
	sentence_indices = Input(shape=input_shape, dtype='int32')
	embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
	embeddings = embedding_layer(sentence_indices)

	# Be careful, the returned output should be a single hidden state, not a batch of sequences.
	#在输出序列中，返回单个 hidden state值还是返回全部time step 的 hidden state值。 False 返回单个， true 返回全部。
	# 128：dimensionality of the output space.
	# if return_sequence=True, it will output all hidden states
	X = LSTM(128, return_sequences=True)(embeddings)
	X = Dropout(0.5)(X)
	X = LSTM(128, return_sequences=False)(X)
	X = Dropout(0.5)(X)
	X = Dense(5)(X)
	X = Activation("softmax")(X)

	model = Model(inputs=sentence_indices, outputs=X)

	return model
#max_len = 5

model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary

from keras.utils import plot_model
import matplotlib.pyplot as plt
plot_model(model, to_file='model.png')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle = True)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

#test
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test_indices)):
	x = X_test_indices
	num = np.argmax(pred[i])
	if(num != Y_test[i]):
		print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

In [35]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

not feeling happy 😞


In [ ]:
###############################################################################################

def read_glove_vecs(glove_file):
	with open(glove_file, 'r') as f:
		words = set()
		word_to_vec_map = {}# dictionary
		for line in f:
			line = line.strip().split()#glove is a file contains word and the corresponding vector
			curr_word = line[0]
			words.add(curr_word)
			word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
		
		i = 1
		words_to_index = {}
		index_to_words = {}
		for w in sorted(words):
			words_to_index[w] = i
			index_to_words[i] = w
			i = i + 1
	return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
	"""Compute softmax values for each sets of scores in x."""
	e_x = np.exp(x - np.max(x))
	return e_x / e_x.sum()


def read_csv(filename = 'data/emojify_data.csv'):
	phrase = []
	emoji = []

	with open (filename) as csvDataFile:
		csvReader = csv.reader(csvDataFile)

		for row in csvReader:
			phrase.append(row[0])
			emoji.append(row[1])

	X = np.asarray(phrase)
	Y = np.asarray(emoji, dtype=int)

	return X, Y

def convert_to_one_hot(Y, C):
	Y = np.eye(C)[Y.reshape(-1)] # will convert every element in Y to a vector
	return Y


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
					"1": ":baseball:",
					"2": ":smile:",
					"3": ":disappointed:",
					"4": ":fork_and_knife:"}

def label_to_emoji(label):
	"""
	Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
	"""
	return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)
			  
	
def print_predictions(X, pred):
	print()
	for i in range(X.shape[0]):
		print(X[i], label_to_emoji(int(pred[i])))
		
		
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
	
	df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
	
	df_conf_norm = df_confusion / df_confusion.sum(axis=1)
	
	plt.matshow(df_confusion, cmap=cmap) # imshow
	#plt.title(title)
	plt.colorbar()
	tick_marks = np.arange(len(df_confusion.columns))
	plt.xticks(tick_marks, df_confusion.columns, rotation=45)
	plt.yticks(tick_marks, df_confusion.index)
	#plt.tight_layout()
	plt.ylabel(df_confusion.index.name)
	plt.xlabel(df_confusion.columns.name)
	
	
def predict(X, Y, W, b, word_to_vec_map):
	"""
	Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
	
	Arguments:
	X -- input data containing sentences, numpy array of shape (m, None)
	Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
	
	Returns:
	pred -- numpy array of shape (m, 1) with your predictions
	"""
	m = X.shape[0]
	pred = np.zeros((m, 1))
	
	for j in range(m):                       # Loop over training examples
		
		# Split jth test example (sentence) into list of lower case words
		words = X[j].lower().split()
		
		# Average words' vectors
		avg = np.zeros((50,))
		for w in words:
			avg += word_to_vec_map[w]
		avg = avg/len(words)

		# Forward propagation
		Z = np.dot(W, avg) + b
		A = softmax(Z)
		pred[j] = np.argmax(A)
		
	print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
	
	return pred